<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/ROC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import requests
import time

# CoinGecko API ile Top 50 Coin'i Çekme
def get_top_coingecko_coins():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        'vs_currency': 'usd',
        'order': 'market_cap_desc',
        'per_page': 50,
        'page': 1,
        'sparkline': False
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return [{'symbol': coin['symbol'].upper(), 'id': coin['id']} for coin in data]
    else:
        print("CoinGecko API'den veri çekilemedi.")
        return []

# CoinGecko'dan Tarihsel Veri Çekme
def get_coingecko_historical_data(coin_id, days=365):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
    params = {
        'vs_currency': 'usd',
        'days': days,
        'interval': 'daily'
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        prices = data['prices']
        df = pd.DataFrame(prices, columns=['timestamp', 'price'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        return df
    else:
        print(f"{coin_id} için veri çekilemedi.")
        return pd.DataFrame()

# Mesafe Metriği Hesaplama Fonksiyonu
def calculate_distance(df):
    close_price = df['price']
    roc = close_price.pct_change(9)  # 9 günlük değişim
    roc_min = roc.rolling(100).min()
    roc_max = roc.rolling(100).max()
    roc_scaled = ((roc - roc_min) / (roc_max - roc_min)) * 8 - 4
    roc_volatility = np.abs(roc.diff()).rolling(9).mean()
    roc_volatility_scaled = ((roc_volatility - roc_volatility.min()) /
                            (roc_volatility.max() - roc_volatility.min())) * 8 - 4
    distance = roc_scaled - roc_volatility_scaled
    return distance.iloc[-1] if not distance.empty else np.nan

# Ana İşlem Akışı
if __name__ == "__main__":
    # Top 50 Coin Listesi
    top_coins = get_top_coingecko_coins()

    # Sonuçları Depolama
    results = []

    # Her Coin için İşlem Yap
    for coin in top_coins:
        symbol = coin['symbol']
        coin_id = coin['id']
        print(f"{symbol} işleniyor...")

        # Tarihsel Veri Çek
        historical_data = get_coingecko_historical_data(coin_id)
        time.sleep(1)  # Rate limit için

        if not historical_data.empty:
            # Mesafe Metriği Hesapla
            distance = calculate_distance(historical_data)

            # Fiyat Bilgisi
            current_price = historical_data['price'].iloc[-1]

            results.append({
                'Coin': symbol,
                'Distance': distance,
                'Price (USD)': current_price
            })

    # Sonuçları Göster
    if results:
        df = pd.DataFrame(results)
        print("\nSonuçlar:")
        print(df.to_string(index=False))
    else:
        print("Hiç veri alınamadı.")

BTC işleniyor...
ETH işleniyor...
XRP işleniyor...
USDT işleniyor...
SOL işleniyor...
solana için veri çekilemedi.
BNB işleniyor...
binancecoin için veri çekilemedi.
USDC işleniyor...
usd-coin için veri çekilemedi.
DOGE işleniyor...
dogecoin için veri çekilemedi.
ADA işleniyor...
cardano için veri çekilemedi.
STETH işleniyor...
staked-ether için veri çekilemedi.
TRX işleniyor...
tron için veri çekilemedi.
LINK işleniyor...
chainlink için veri çekilemedi.
WBTC işleniyor...
wrapped-bitcoin için veri çekilemedi.
AVAX işleniyor...
avalanche-2 için veri çekilemedi.
WSTETH işleniyor...
wrapped-steth için veri çekilemedi.
XLM işleniyor...
stellar için veri çekilemedi.
TON işleniyor...
the-open-network için veri çekilemedi.
SUI işleniyor...
sui için veri çekilemedi.
HBAR işleniyor...
hedera-hashgraph için veri çekilemedi.
SHIB işleniyor...
shiba-inu için veri çekilemedi.
WETH işleniyor...
weth için veri çekilemedi.
LEO işleniyor...
leo-token için veri çekilemedi.
DOT işleniyor...
polkadot için